In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm


## Leer CSV

In [ ]:
try:
    df_results = pd.read_csv("resultados_satt_todos.csv")
except FileNotFoundError:
    exit()
except Exception as e:
    print(f"Error al leer el CSV: {e}")
    try:
        df_results = pd.read_csv("resultados_satt_todos.csv", delimiter=';')
        print("Leído exitosamente con delimitador punto y coma.")
    except Exception as e_semi:
        print(f"Error al leer con punto y coma también: {e_semi}")
        exit()
        
df_results.columns = df_results.columns.str.strip()

## Pre-tratamiento de los datos

In [ ]:
# Convertir columnas numéricas
numeric_cols = ['satt_true', 'satt_est', 'bias', 'mse', 'ci_ancho', 'tiempo_seg']
for col in numeric_cols:
    if col in df_results.columns:
        df_results[col] = pd.to_numeric(df_results[col], errors='coerce')

# Convertir 'cobertura_ic' a booleano
if 'cobertura_ic' in df_results.columns:
    if df_results['cobertura_ic'].dtype == 'object':
        df_results['cobertura_ic'] = df_results['cobertura_ic'].astype(str).str.strip().str.lower().map({'true': True, 'false': False})
    elif pd.api.types.is_numeric_dtype(df_results['cobertura_ic']):
        df_results['cobertura_ic'] = df_results['cobertura_ic'].astype(bool)
    df_results.dropna(subset=['cobertura_ic'], inplace=True)
    if not pd.api.types.is_bool_dtype(df_results['cobertura_ic']):
         print("Advertencia: La columna 'cobertura_ic' contiene valores no booleanos después de la conversión.")
else:
    print("Advertencia: Columna 'cobertura_ic' no encontrada.")

# Eliminar filas donde las métricas numéricas clave son NaN para los cálculos de promedio
df_cleaned_numeric = df_results.dropna(subset=[col for col in ['bias', 'mse', 'ci_ancho'] if col in df_results.columns])

In [ ]:
print(f"Filas en df_results: {len(df_results)}")
print(f"Filas en df_cleaned_numeric: {len(df_cleaned_numeric)}")

# Verificar si hay NaNs en las columnas clave DESPUÉS de la conversión a numérico
print("\nNaNs en df_results después de to_numeric (para columnas clave del dropna):")
print(df_results[['bias', 'mse', 'ci_ancho']].isnull().sum())

if len(df_results) == len(df_cleaned_numeric):
    print("\nConfirmado: df_results y df_cleaned_numeric tienen el mismo número de filas.")
    print("Esto implica que no había NaNs en 'bias', 'mse', o 'ci_ancho' en df_results (después de to_numeric).")
else:
    print("\nAdvertencia: df_results y df_cleaned_numeric tienen diferente número de filas.")
    print("Esto implica que SÍ había NaNs en 'bias', 'mse', o 'ci_ancho' que fueron eliminados.")

## Análisis

In [ ]:
print("\n=== Resumen Global Calculado ===")
total_datasets_in_file = len(df_results)
total_datasets_for_numeric_avg = len(df_cleaned_numeric)

cobertura_media_calc = df_results['cobertura_ic'].mean() * 100 if 'cobertura_ic' in df_results.columns and pd.api.types.is_bool_dtype(df_results['cobertura_ic']) else np.nan
ancho_medio_calc = df_cleaned_numeric['ci_ancho'].mean() if 'ci_ancho' in df_cleaned_numeric.columns else np.nan
bias_medio_calc = df_cleaned_numeric['bias'].mean() if 'bias' in df_cleaned_numeric.columns else np.nan
mse_medio_calc = df_cleaned_numeric['mse'].mean() if 'mse' in df_cleaned_numeric.columns else np.nan
# RMSE global
rmse_global = np.sqrt(df_results['mse'].mean()) if 'mse' in df_results.columns else np.nan


print(f"Datasets procesados (después de limpiar cobertura_ic): {total_datasets_in_file}")
print(f"Datasets usados para promedios numéricos: {total_datasets_for_numeric_avg}")
print(f"Cobertura IC        : {cobertura_media_calc:.2f} %")
print(f"Ancho IC            : {ancho_medio_calc:.4f}")
print(f"Promedio Bias       : {bias_medio_calc:.4f}")
print(f"Promedio MSE        : {mse_medio_calc:.4f}")
print(f'RMSE global         : {rmse_global:.4f}')


if 'modelo_elegido' in df_results.columns:
    print("\nModelos elegidos (sobre datos con cobertura_ic válida):")
    print(df_results['modelo_elegido'].value_counts())

In [ ]:
# Agregado completo por modelo
agg_modelo = (
    df_results.groupby('modelo_elegido')
      .agg(
          n_archivos = ('archivo',   'size'),
          rmse       = ('mse',       lambda x: np.sqrt(x.mean())),
          bias       = ('bias',      'mean'),
          cobertura  = ('cobertura_ic', 'mean'),
          ancho_ic   = ('ci_ancho',  'mean'),
          tiempo_seg = ('tiempo_seg','mean')
      )
      .reset_index()
      .sort_values('modelo_elegido')
)

agg_modelo['cobertura_%'] = agg_modelo['cobertura'] * 100
agg_modelo.drop(columns='cobertura', inplace=True)

print("\n--- Rendimiento promedio por modelo ---")
print(
    agg_modelo[
        ['modelo_elegido','bias','rmse','cobertura_%','ancho_ic','tiempo_seg']
    ].round(4)
)


In [ ]:
# RMSE por carpeta
rmse_carpeta = (
    df_results.groupby('carpeta')['mse']
      .mean()
      .pipe(np.sqrt)
)
rmse_carpeta.head()

In [ ]:
# Agregado completo por carpeta
agg_carpeta = (
    df_results.groupby('carpeta')
      .agg(
          n_archivos = ('archivo', 'size'),
          rmse       = ('mse',     lambda x: np.sqrt(x.mean())),
          bias       = ('bias',    'mean'),
          cobertura  = ('cobertura_ic', 'mean'),
          ancho_ic   = ('ci_ancho',     'mean')
      )
      .reset_index()
      .sort_values('carpeta')
)

# Rendimiento promedio por carpeta
agg_carpeta['cobertura_%'] = agg_carpeta['cobertura'] * 100

print("\n--- Rendimiento promedio por carpeta ---")
print(
    agg_carpeta[
        ['carpeta', 'bias', 'rmse', 'cobertura_%', 'ancho_ic', 'n_archivos']
    ].round({'bias':4, 'rmse':4, 'cobertura_%':2, 'ancho_ic':4})
)

In [ ]:
# TOP -> B según RMSE
print("\nCarpetas con mayor RMSE promedio:")
print(agg_carpeta.nlargest(5, 'rmse')[['carpeta','rmse']].round(4))

print("\nCarpetas con menor RMSE promedio:")
print(agg_carpeta.nsmallest(5, 'rmse')[['carpeta','rmse']].round(4))

# TOP -> B según Cobertura
print("\nCarpetas con menor Cobertura IC promedio (%):")
print(agg_carpeta.nsmallest(5, 'cobertura_%')[['carpeta','cobertura_%']].round(2))

# TOP -> B según Ancho de IC
print("\nCarpetas con mayor Ancho IC promedio:")
print(agg_carpeta.nlargest(5, 'ancho_ic')[['carpeta','ancho_ic']].round(4))


## Test de hipótesis

Comparativa de la submuestra con el análisis del CalCause original (Dorie et al., 2019, Fig. 3)

In [ ]:
# Parámetros observados
n     = len(df_results)
x     = df_results['cobertura_ic'].sum()  # casos en que el IC cubre el SATT
p_hat = x / n

# Benchmark del paper CalCause
p_0 = 0.82

# Estadístico Z
se   = np.sqrt(p_0 * (1 - p_0) / n)
z    = (p_hat - p_0) / se
pval = 2 * (1 - norm.cdf(abs(z)))

print(f"Cobertura observada: {p_hat:.4f}")
print(f"Cobertura esperada (CalCause): {p_0}")
print(f"Z = {z:.3f}")
print(f"p-valor = {pval:.4g}")